# Lesson 05: wind flow, 4D data, numerical derivatives

In this exercise we will learn to apply new plotting functions and learn a bit more about the underlying details of the tools we are using.

## Import the packages

This did not change:

In [ ]:
# Define the tools we are going to need today
%matplotlib inline
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xray  # NetCDF library
import cartopy  # Plotting libary
import cartopy.crs as ccrs  # Projections
# Some defaults
plt.rcParams['figure.figsize'] = (14, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are not important
import warnings
warnings.filterwarnings('ignore')

## Read the data

Today we are going to use a new data file, ``ERA-Int-MonthlyAvg-4D-UVWZ.nc``. As always you will find it on OLAT or in the scratch directory(``/scratch/c707/c7071047/data``). Open the file and explore it.

**Q: what are the dimensions of this data? What is new/different in comparison to the previous files? What are the variables available in the file, what are their units? What is the variable "level"?**

In [ ]:
netcdf = xray.open_dataset('ERA-Int-MonthlyAvg-4D-UVWZ.nc')

## Plotting wind fields

We are now reading the u and v windfields at the 1000hPa level for the month of January:

In [ ]:
u = netcdf.u.sel(level=1000, month=1).copy()
v = netcdf.v.sel(level=1000, month=1).copy()

### Quiver plots 

One traditional way to plot the wind data is as vector arrows. We can use a function called [quiver()](http://matplotlib.org/api/pyplot_api.html?highlight=quiver#matplotlib.pyplot.quiver) for that:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
pu, pv = u[::9,::9], v[::9,::9]  # we will discuss what this line does in the cells below
qv = ax.quiver(pu.longitude, pu.latitude, pu, pv, transform=ccrs.PlateCarree())
ax.coastlines();

Note the presence of the line ``pu, pv = u[::9,::9], v[::9,::9]``. Let's explain it in detail. the ``::9`` is the Numpy way to index each 9th element of an array. It is called [slicing](http://docs.scipy.org/doc/numpy-1.10.0/reference/arrays.indexing.html#basic-slicing-and-indexing) in the numpy jargon. Let's try slicing on a simpler array:

In [ ]:
a = np.arange(11)
print(a)

**E: try and explain the following commands:**
- a[:]
- a[2:]
- a[:-1]
- a[2:5]
- a[::-1]
- a[::2]
- a[::-2]

In [ ]:
# your answer here

Ok, so that was slincing in one dimensions. Slicing also works in N-dimensions. For example:

In [ ]:
a = np.arange(4*5).reshape((4, 5))
print(a)

**E: try and explain the following commands:**
- a[:, ::-1]
- a[::2, ::2]
- a[::2, ::3]

In [ ]:
# your answer here

Ok, that was slicing in two dimensions, so we now better understand what ``u[::9,::9]`` means. But what was about that comma in between? This is simply what people call "syntaxic sugar", a nice and easy way to write one line in two: 

In [ ]:
a, b = 12, 'Hello'
print(a)
print(b)

This kind of shortcuts should be used only if:
- the lines of code are easy
- the two variables are somehow related (for example u and v winds)

If you abuse of this kind of "one-liners", people reading your code will hate you. 

**E: reproduce the quiver plot but with other slices. For example 5 or 15. What happens if you do no slicing at all?**

In [ ]:
# your answer here

**E: compute the wind speed out of u and v. Plot the wind speed as a shaded color plot (as we did for temperature and precipitation) and plot the wind arrows on top of it.**

In [ ]:
# your answer here

### Streamlines 

Another way to plot wind data are the so called [streamlines](http://matplotlib.org/api/pyplot_api.html?highlight=streamplot#matplotlib.pyplot.streamplot):

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.streamplot(u.longitude, u.latitude, u.values, v.values, transform=ccrs.PlateCarree(), 
              density=4)
ax.coastlines();

*Note: if this is too slow for your taste, you can slice your data like we did for the quiver plot, this will produce the plot faster*

A problem with streamlines is that they provide no information about the strength of the flow. It is possible to display this information with colors: 

In [ ]:
ws = (u**2 + v**2)**0.5
ax = plt.axes(projection=ccrs.PlateCarree())
strm = ax.streamplot(u.longitude, u.latitude, u.values, v.values, transform=ccrs.PlateCarree(),
              density=4, color=ws.values, cmap=plt.get_cmap('cool'))
plt.colorbar(strm.lines)
ax.coastlines()
ax.gridlines(draw_labels=True);  # what is this line doing?

**E: plot the wind streamlines for the month of july. Discuss the differences in the flow. Where it the ITCZ located for both months? Are there other strong differences?**

In [ ]:
# your answer here

## Upper atmosphere data 

We are going to analyse further plots of the upper troposphere atmospheric circulation in exercise 06. In today's lesson we are just going to note that it can be really easy to make zonal plots of the vertical structure of the atmopshere:

In [ ]:
u_all = netcdf.u.sel(month=1).copy()
u_all = u_all.mean(dim='longitude')

In [ ]:
u_all.plot();

Ugh. This is not soooo easy as I thought. Here's how to make it a bit better:

In [ ]:
u_all.name = 'U-wind (m s-1)'
u_all.plot.contourf(levels=np.linspace(-45, 45, 19));
plt.ylim([1000, 50]);
plt.ylabel('Pressure (hPa)');

This linear representation of the atmosphere is often preferred and looks familiar. However, to represent more acurately the real altitude of the winds, a logarithmic scale might be better:

In [ ]:
u_all.plot.contourf(levels=np.linspace(-45, 45, 19));
plt.ylim([1000, 50])
plt.yscale('log')
plt.ylabel('Pressure (hPa)');

With a logarithmic scale the importance of the westerly winds for most parts of the atmosphere is clearly visible.

## Some details you need to know before going on

I hope that after mesing around with python a couple of hours you are now convinced that it is a very intuitive language. In particular, the creators of python put a lot of efforts in the *readability* of it syntax, which is appreciated by more and more scientists worldwide. The simplicity with which we produced rather complex analyses is the best proof of its success.

When analyses become a bit more complex we cannot afford to ignore too many details about the internals of the libraries and functions we are using. The next couple of cells are going to explain a little bit what is happening "under the hood" when we analyse our data.

#### Some details about netcdf files

We are going to start with the very begining again:

In [ ]:
netcdf = xray.open_dataset('ERA-Int-MonthlyAvg-4D-UVWZ.nc')

The ``netcdf`` variable is a link to a file on disk. Being able to "explore" it, listing its variables and printing them does not mean that the data has been *read* out of the file. This is a very usefull property of the netcdf tool: without this property it would be impossible to open large data files that do not fit in memory. Even more interesting, "reading" a variable will *not* read the data out of the file:

In [ ]:
z = netcdf.z
z

See what is printed out: ``[20822400 values with dtype=float64]``. In other words, the data has not yet been read. It is going to be read only if really needed:

In [ ]:
z = z.sel(month=1, level=500)
z

Still not read. But if I compute something:

In [ ]:
z = z / 9.81
z

Now it was necessary to read the data in order to do the required computation.

#### Some details about the type of variables we are using 

Python has a built-in function which says the type of a variable:

In [ ]:
type(1)

In [ ]:
type(3.14)

In [ ]:
type(z)

In [ ]:
a = np.arange(10)
type(a)

These are many details, but very often reading the first and the last word of the variable type provides enough information: 
- "z" is a variable of type "DataArray" provided by the "xray" library. 
- "a" is a variable of type "ndarray" provided by the "numpy" library.

Both xray and numpy are python ``libraries``. Both are open-source packages (see [here](https://github.com/numpy/numpy) and [here](https://github.com/xray/xray)). Xray is younger than Numpy and builds upon it (xray needs numpy to work). Xray's DataArray are very similar to numpy arrays but they are more powerful. Under the hood, xray's DataArray are nothing more then a numpy ndarray. Let's see:

In [ ]:
type(z.values)

So the data we are working with is internally stored in a numpy array. The reason for us to use xray on top of numpy is because xray is more user friendly. Try to plot a numpy array for example:

In [ ]:
z_array = z.values

In [ ]:
z_array.plot()

Or try to select some slices of longitude:

In [ ]:
z_array.sel(longitude=slice(20, 90))

Numpy arrays are only indexable through indices:

In [ ]:
z_array.shape

In [ ]:
z_array[12:14, 34:36]

Which is much less "friendly" than the selection by dimension and coordinate names offered by xray. Xray adds names ("labels") and coordinates to multi-dimenional data.

The reason why I explained these differences between xray and numpy is that sometimes we have to leave xray and go back to numpy functions for certain computations, as we shall see below. 

## Numerical derivatives of gridded data 

In this lesson we are going to learn how to compute the derivative of a meteorological field. One of the easiest formulas available to us is the geostrophic wind equation. We will compute geostrophic winds out of the geopotential field.

The geostrophic equation can be expressed in pressure coordinates as:

$$u_g = -\frac{g}{f} \frac{\partial z}{\partial y}$$

$$v_g = \frac{g}{f} \frac{\partial z}{\partial x}$$

with $u_g$, $v_g$ the geostrophic wind components and z the geopotential (in m).


First we need to compute the corolis parameter $f = 2  \Omega \sin \varphi$:

In [ ]:
f = 2. * 7.292115e-5 * np.sin(np.deg2rad(u.latitude))

**Q: what is the type of f? Plot it.**

In [ ]:
# your answer here

To avoid some mathematical singularities we are going to mask out the areas of the globe where trigonometry becomes too close to zero: 

In [ ]:
f = f.where((np.abs(u.latitude) > 5) & (np.abs(u.latitude) < 85))

#### Gradient of a 2d field

Computing the [gradient of discrete data](https://en.wikipedia.org/wiki/Finite_difference_method) is something computers know how to do. For the details of the algorithm you can refer to other courses in numerical methods. Here we are going to use the function [gradient()](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.gradient.html) provided by numpy:

In [ ]:
z = netcdf.z.sel(month=1, level=500) / 9.81

In [ ]:
grad_y, grad_x = np.gradient(z, -np.deg2rad(.75), np.deg2rad(.75))

**Q: read the documentation of the np.gradient function. Explain why I needed to provide -np.deg2rad(.75) and np.deg2rad(.75) as arguments. What is the type of grad_y and grad_x? Why are they returned by the function in that order? What is their shape?**

Because np.gradient() is not giving us xray DataArray variables. We are going to use a trick to convert them back: 

In [ ]:
grad_x = z*0 + grad_x
grad_y = z*0 + grad_y

There are other ways to do what we just did above, but this is clearly the easiest (and therefore: the best).

**Q: can you try to explain how the conversion occured? Now plot the variables grad_x and grad_y**

In [ ]:
# Your answer here

Before moving foreward I have to draw your attention towards a certain detail. The equations of geostrophic winds described above are provided in *cartesian* (x, y) coordinates. We just computed the gradient in *spherical* (lon, lat) coordinates. Once again, we have to remember that the Earth is not flat and take this effect into account:

$$\frac{\partial}{\partial x} = \frac{1}{R \cos \varphi}\frac{\partial}{\partial\lambda}$$

$$\frac{\partial}{\partial y} = \frac{1}{R}\frac{\partial}{\partial\varphi}$$

with $\lambda$, $\varphi$ the longitude and latitude and R the Earth' radius. We apply the formulas and define a factor for the spherical derivatives:

In [ ]:
dx = 1 / (6371000 * np.cos(np.deg2rad(z.latitude)))
dy = 1 / (6371000)

Now we are ready to compute the geostrophic winds:

In [ ]:
g = 9.81

In [ ]:
ug = - g / f * dy * grad_y
vg = g / f * dx * grad_x

In [ ]:
ws = (ug**2 + vg**2)**0.5
ax = plt.axes(projection=ccrs.PlateCarree())
strm = ax.streamplot(ug.longitude, ug.latitude, ug.values, vg.values, transform=ccrs.PlateCarree(),
              density=4, color=ws.values, cmap=plt.get_cmap('cool'))
plt.colorbar(strm.lines)
ax.coastlines()
ax.gridlines(draw_labels=True);

**E: compare our computed geostrophic wind with the "real" wind from the reanalysis data at the same level and month. Are there large differences? Where are the wind speed differences largest? What are the possible reasons for these differences?** Hint: you can plot the wind speed differences on a map too.

In [ ]:
# your answer here

## What next?

For those who have't finished the exercises 04 you can go back to them. Otherwise go to exercise 6.